#Importing Data and Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import glob
from IPython.display import display

##remove emoji only text

In [ ]:
# import re

# # Function to check if a string contains only emojis
# def contains_only_emojis(text):
#     emoji_pattern = re.compile("["
#                                u"\U0001F600-\U0001F64F"  # emoticons
#                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                                u"\U00002700-\U000027BF"  # Dingbats
#                                u"\U000024C2-\U0001F251"
#                                "]+", flags=re.UNICODE)
#     # If the text contains only emojis, the text without emojis should be empty
#     text_without_emojis = emoji_pattern.sub(r'', text)
#     return len(text_without_emojis.strip()) == 0

# # Apply the filter function and drop rows with only emojis
# df_filtered = df[~df['text'].apply(contains_only_emojis)]

# # Save the filtered DataFrame back to a new CSV file
# df_filtered.to_csv('./drive/MyDrive/TA Tugas Akhir Workspace/data_training.csv', index=False)

# print("Filtered data saved to 'data_training.csv'")


In [ ]:
# df['text'] = df['combined_Text']
# df = df.rename(columns={'combined_Text': 'text_origin'})
# df.to_csv('./drive/MyDrive/TA Tugas Akhir Workspace/data_training.csv', index=False)
# df.head()

##Read Labelled File

In [ ]:
dflabel = pd.read_excel('./drive/MyDrive/TA Tugas Akhir Workspace/Dataset/Train/labelled_data.xlsx')
dflabel.head()

,text,sentiment
0,keluar nya jauh bgt,negative
1,"seribu kenangan, walaupun cape jalan keluar ny...",negative
2,candi prambanan aku titip kenanganku ditempatm...,neutral
3,"kangen bgt sama jogja, apalgi kenangan nya.",positive
4,"aku, kamu, Prambanan",neutral


In [ ]:
dflabel = dflabel.rename(columns={'text': 'text_origin'})
dflabel['text'] = dflabel['text_origin']
dflabel.head()

,text_origin,sentiment,text
0,keluar nya jauh bgt,negative,keluar nya jauh bgt
1,"seribu kenangan, walaupun cape jalan keluar ny...",negative,"seribu kenangan, walaupun cape jalan keluar ny..."
2,candi prambanan aku titip kenanganku ditempatm...,neutral,candi prambanan aku titip kenanganku ditempatm...
3,"kangen bgt sama jogja, apalgi kenangan nya.",positive,"kangen bgt sama jogja, apalgi kenangan nya."
4,"aku, kamu, Prambanan",neutral,"aku, kamu, Prambanan"


In [ ]:
dflabel = dflabel[['text_origin','text','sentiment']]
dflabel.head()
# dflabel.to_csv('./drive/MyDrive/TA Tugas Akhir Workspace/labelled_combined_text.csv', index=False)

,text_origin,text,sentiment
0,keluar nya jauh bgt,keluar nya jauh bgt,negative
1,"seribu kenangan, walaupun cape jalan keluar ny...","seribu kenangan, walaupun cape jalan keluar ny...",negative
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral
3,"kangen bgt sama jogja, apalgi kenangan nya.","kangen bgt sama jogja, apalgi kenangan nya.",positive
4,"aku, kamu, Prambanan","aku, kamu, Prambanan",neutral


In [ ]:
dflabel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text_origin  3035 non-null   object
 1   text         3035 non-null   object
 2   sentiment    3035 non-null   object
dtypes: object(3)
memory usage: 71.3+ KB


##Count Distinct Label

In [ ]:
print(dflabel['sentiment'].value_counts())

sentiment
positive    1328
neutral     1220
negative     487
Name: count, dtype: int64


# Preprocessing

## Removing Null data and dropping 'confident score' column

In [ ]:
dflabel.isnull().sum()
dflabel.dropna(inplace=True)
dflabel.drop_duplicates()
dflabel.head()

,text_origin,text,sentiment
0,keluar nya jauh bgt,keluar nya jauh bgt,negative
1,"seribu kenangan, walaupun cape jalan keluar ny...","seribu kenangan, walaupun cape jalan keluar ny...",negative
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral
3,"kangen bgt sama jogja, apalgi kenangan nya.","kangen bgt sama jogja, apalgi kenangan nya.",positive
4,"aku, kamu, Prambanan","aku, kamu, Prambanan",neutral


##Lowercase all text

In [ ]:
dflabel["text"] = dflabel["text"].apply(lambda item: str(item).lower())
dflabel.head()

,text_origin,text,sentiment
0,keluar nya jauh bgt,keluar nya jauh bgt,negative
1,"seribu kenangan, walaupun cape jalan keluar ny...","seribu kenangan, walaupun cape jalan keluar ny...",negative
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral
3,"kangen bgt sama jogja, apalgi kenangan nya.","kangen bgt sama jogja, apalgi kenangan nya.",positive
4,"aku, kamu, Prambanan","aku, kamu, prambanan",neutral


In [ ]:
import re

def remove_special_text(text):
    text = text.replace('\t', ' ').replace('\n', ' ')

    text = text.encode('ascii', 'replace').decode('ascii')

    text = re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text)

    text = ' '.join(text.split())

    return text


In [ ]:
dflabel['text'] = dflabel['text'].astype('str')
dflabel['text'] = dflabel['text'].apply(remove_special_text)
dflabel['text'].unique()

array(['keluar nya jauh bgt',
       'seribu kenangan, walaupun cape jalan keluar nya anjir???',
       'candi prambanan aku titip kenanganku ditempatmu waktu study tour',
       ..., 'nggilani', 'bosen jeng tampilanmu cuman itu itu aja',
       'ayo ras'], dtype=object)

In [ ]:
import string
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
def remove_whitespace(text):
    return text.strip()
def remove_multispace(text):
    return re.sub('\s+',' ',text)
def spell(word):
    if re.search(r'([a-zA-Z])\1{2,}', word):
        rword = re.sub(r'([a-zA-Z])\1{2,}', r'\1', word)
        return rword
    else:
        return word


In [ ]:
dflabel['text'] = dflabel['text'].apply(remove_punctuation)
dflabel['text'] = dflabel['text'].apply(remove_whitespace)
dflabel['text'] = dflabel['text'].apply(remove_multispace)
dflabel['text'] = dflabel['text'].apply(spell)
dflabel['text'].unique()

array(['keluar nya jauh bgt',
       'seribu kenangan walaupun cape jalan keluar nya anjir',
       'candi prambanan aku titip kenanganku ditempatmu waktu study tour',
       ..., 'nggilani', 'bosen jeng tampilanmu cuman itu itu aja',
       'ayo ras'], dtype=object)

In [ ]:
dflabel.head()

,text_origin,text,sentiment
0,keluar nya jauh bgt,keluar nya jauh bgt,negative
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral


In [ ]:
import nltk
from nltk.tokenize import word_tokenize


In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

def tokenize(word):
  word = word.split(" ")
  return word


In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dflabel['text_tokenized'] = dflabel['text'].apply(word_tokenize_wrapper)
dflabel.head()

,text_origin,text,sentiment,text_tokenized
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]"
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu..."
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit..."
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]"
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]"


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(['yg', 'dgn', 'dg', 'dlm', 'dr',
    'tp', 'bgt', 'jg', 'ga', 'tdk',
    'klo', 'gw', 'lu', 'gak', 'emg',
    'jd', 'gmn', 'dpt', 'bs', 'utk','sllu','anjir','nya'])
other_stopword = pd.read_csv('./drive/MyDrive/TA Tugas Akhir Workspace/stopwords-id322.txt',names= ["stopwords"], header = None)
list_stopwords.extend(other_stopword['stopwords'][0].split(' '))
list_stopwords = set(list_stopwords)
exception_words = set(['jauh','dekat','besar','kecil', 'keluar','masuk'])
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords or word in exception_words]


In [ ]:
# other_stopword = pd.read_csv('./drive/MyDrive/TA Tugas Akhir Workspace/stopwords-id322.txt',names= ["stopwords"], header = None)
# list_stopwords = set(other_stopword['stopwords'][0].split(' '))
# def stopwords_removal(words):
#     return [word for word in words if word not in list_stopwords]


In [ ]:
dflabel['text_tokenized_cleaned'] = dflabel['text_tokenized'].apply(stopwords_removal)
dflabel.head()

,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]"
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]"
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat..."
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]"
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan]


In [ ]:
!pip install Sastrawi
!pip install swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


In [ ]:
df_normalized = pd.DataFrame(dflabel['sentiment'])
df_normalized['text'] = dflabel['text']

In [ ]:
normalized_word = df_normalized
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

dflabel['text_normalized'] = dflabel['text_tokenized_cleaned'].apply(normalized)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}

for document in dflabel['text_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")


Streaming output truncated to the last 5000 lines.
dirugikan : rugi
alasan : alas
mengiranya : ira
meraup : raup
ratusan : ratus
ribubhkn : ribubhkn
jutaan : juta
pemikiran : pikir
membandingkan : banding
bersaing : saing
darat : darat
angkotbus : angkotbus
opang : opang
1jt : 1jt
pergerakan : gera
matiitu : matiitu
kapok : kapok
wajarapalagi : wajarapalagi
contohnya : contoh
percari : cari
terkena : kena
dampaknya : dampak
akibat : akibat
sugesti : sugesti
ongkosnya : ongkos
nggadek : nggadek
geleman : geman
wajarlah : wajar
carinya : cari
wonge : wonge
abis : abis
jngn : jngn
ngarit : ngarit
dimakan : makan
selangit : langit
habiskan : habis
nol : nol
100rb : 100rb
lha : lha
jaranne : jaranne
open : open
openane : openane
jonz : jonz
erte : erte
nggeh : nggeh
nopo2 : nopo2
gendong : gendong
piro : piro
angut : angut
jeef : jeef
parang : parang
tritis : tritis
300 : 300
patungan : patungan
rbkarena : rbkarena
angkut : angkut
murahgak : murahgak
sepakat : sepakat
kenyataan : nyata
pesa

In [ ]:
dflabel.head()

,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned,text_normalized
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]","[keluar, jauh]"
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]","[seribu, kenangan, cape, jalan, keluar]"
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenanganku, ditempat..."
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenangan]"
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan],[prambanan]


In [ ]:
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

dflabel['text_stemmed'] = dflabel['text_normalized'].swifter.apply(get_stemmed_term)

Pandas Apply:   0%|          | 0/3035 [00:00<?, ?it/s]

In [ ]:
def merge_again(text):
    return ' '.join(text)
dflabel['text_final'] = dflabel['text_stemmed'].apply(merge_again)



In [ ]:
dflabel.head()

,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned,text_normalized,text_stemmed,text_final
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]","[keluar, jauh]","[keluar, jauh]",keluar jauh
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]","[seribu, kenangan, cape, jalan, keluar]","[ribu, kenang, cape, jalan, keluar]",ribu kenang cape jalan keluar
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenang, tempat, stud...",candi prambanan titip kenang tempat study tour
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenang]",kangen jogja apalgi kenang
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan],[prambanan],[prambanan],prambanan


In [ ]:
dflabel.to_csv('./drive/MyDrive/TA Tugas Akhir Workspace/training_data.csv', index=False)

# Encode Nonnumerical Data


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# # dflabel["Sentiment"] = label_encoder.fit_transform(dflabel[["Sentiment"]])
# # dflabel.head()


In [ ]:
df_final = dflabel.copy()
df_final.head()

,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned,text_normalized,text_stemmed,text_final
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]","[keluar, jauh]","[keluar, jauh]",keluar jauh
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]","[seribu, kenangan, cape, jalan, keluar]","[ribu, kenang, cape, jalan, keluar]",ribu kenang cape jalan keluar
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenang, tempat, stud...",candi prambanan titip kenang tempat study tour
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenang]",kangen jogja apalgi kenang
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan],[prambanan],[prambanan],prambanan


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd



X = df_final['text_final']
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)
tfidf_vectors = X_tfidf.toarray().tolist()
df_final['tfidf_vector'] = tfidf_vectors
df_final.head(6)


,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned,text_normalized,text_stemmed,text_final,tfidf_vector
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]","[keluar, jauh]","[keluar, jauh]",keluar jauh,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]","[seribu, kenangan, cape, jalan, keluar]","[ribu, kenang, cape, jalan, keluar]",ribu kenang cape jalan keluar,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenang, tempat, stud...",candi prambanan titip kenang tempat study tour,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenang]",kangen jogja apalgi kenang,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan],[prambanan],[prambanan],prambanan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,kangen bnget jogjaa,kangen bnget jogjaa,positive,"[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]",kangen bnget jogjaa,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
print(dflabel['sentiment'].value_counts())

sentiment
positive    1328
neutral     1220
negative     487
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X = dflabel['text_final']
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(X)
# # Convert the TF-IDF matrix to a list of vectors
# tfidf_vectors = X_tfidf.toarray().tolist()

# # Add the vectors as a new column in the DataFrame
# dflabel['tfidf_vector'] = tfidf_vectors

# # Display the first few rows of the DataFrame
# dflabel.head()


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, make_scorer
from imblearn.over_sampling import SMOTE

nb_classifier = MultinomialNB()

X = dflabel['text_final']
y = dflabel['sentiment']




In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1, shuffle=True)
print(y.value_counts())

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100}%")

print("Classification Report:")
print(classification_report(y_test, y_pred))



sentiment
positive    1328
neutral     1220
negative     487
Name: count, dtype: int64
Accuracy: 64.41515650741351%
Classification Report:
              precision    recall  f1-score   support

    negative       1.00      0.07      0.12        90
     neutral       0.63      0.62      0.63       256
    positive       0.64      0.87      0.74       261

    accuracy                           0.64       607
   macro avg       0.76      0.52      0.50       607
weighted avg       0.69      0.64      0.60       607



In [ ]:
feature_names = tfidf_vectorizer.get_feature_names_out()
X_dense = X_tfidf.todense()
df_tfidf = pd.DataFrame(X_dense, columns=feature_names)
def extract_tfidf_scores(text, tfidf_df):
    words = text.split()
    return {word: tfidf_df.at[i, word] for i, word in enumerate(words) if word in tfidf_df.columns}

df_final['tfidf_vector'] = df_final['text_final'].apply(lambda x: extract_tfidf_scores(x, df_tfidf))
df_final.head(15)

,text_origin,text,sentiment,text_tokenized,text_tokenized_cleaned,text_normalized,text_stemmed,text_final,tfidf_vector
0,keluar nya jauh bgt,keluar nya jauh bgt,negative,"[keluar, nya, jauh, bgt]","[keluar, jauh]","[keluar, jauh]","[keluar, jauh]",keluar jauh,"{'keluar': 0.7347135606825228, 'jauh': 0.0}"
1,"seribu kenangan, walaupun cape jalan keluar ny...",seribu kenangan walaupun cape jalan keluar nya...,negative,"[seribu, kenangan, walaupun, cape, jalan, kelu...","[seribu, kenangan, cape, jalan, keluar]","[seribu, kenangan, cape, jalan, keluar]","[ribu, kenang, cape, jalan, keluar]",ribu kenang cape jalan keluar,"{'ribu': 0.0, 'kenang': 0.3813280994016298, 'c..."
2,candi prambanan aku titip kenanganku ditempatm...,candi prambanan aku titip kenanganku ditempatm...,neutral,"[candi, prambanan, aku, titip, kenanganku, dit...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenanganku, ditempat...","[candi, prambanan, titip, kenang, tempat, stud...",candi prambanan titip kenang tempat study tour,"{'candi': 0.0, 'prambanan': 0.0, 'titip': 0.43..."
3,"kangen bgt sama jogja, apalgi kenangan nya.",kangen bgt sama jogja apalgi kenangan nya,positive,"[kangen, bgt, sama, jogja, apalgi, kenangan, nya]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenangan]","[kangen, jogja, apalgi, kenang]",kangen jogja apalgi kenang,"{'kangen': 0.0, 'jogja': 0.0, 'apalgi': 0.0, '..."
4,"aku, kamu, Prambanan",aku kamu prambanan,neutral,"[aku, kamu, prambanan]",[prambanan],[prambanan],[prambanan],prambanan,{'prambanan': 0.0}
5,kangen bnget jogjaa,kangen bnget jogjaa,positive,"[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]","[kangen, bnget, jogjaa]",kangen bnget jogjaa,"{'kangen': 0.0, 'bnget': 0.0, 'jogjaa': 0.0}"
6,bisa ga ya study tour part 2 ke prambanan ? :(,bisa ga ya study tour part 2 ke prambanan,neutral,"[bisa, ga, ya, study, tour, part, 2, ke, pramb...","[ya, study, tour, part, 2, prambanan]","[ya, study, tour, part, 2, prambanan]","[ya, study, tour, part, 2, prambanan]",ya study tour part 2 prambanan,"{'ya': 0.0, 'study': 0.0, 'tour': 0.4016455404..."
7,"aku, teman"" smp dan candi prambanan",aku teman smp dan candi prambanan,neutral,"[aku, teman, smp, dan, candi, prambanan]","[teman, smp, candi, prambanan]","[teman, smp, candi, prambanan]","[teman, smp, candi, prambanan]",teman smp candi prambanan,"{'teman': 0.0, 'smp': 0.0, 'candi': 0.26832315..."
8,"aku kamu dan candi prambanan kala itu""sekarang...",aku kamu dan candi prambanan kala itusekarang ...,neutral,"[aku, kamu, dan, candi, prambanan, kala, ituse...","[candi, prambanan, itusekarang, asing]","[candi, prambanan, itusekarang, asing]","[candi, prambanan, itusekarang, asing]",candi prambanan itusekarang asing,"{'candi': 0.0, 'prambanan': 0.0, 'itusekarang'..."
9,disitunya cuma sebentar tapi kenangannya ga se...,disitunya cuma sebentar tapi kenangannya ga se...,positive,"[disitunya, cuma, sebentar, tapi, kenangannya,...","[disitunya, sebentar, kenangannya, sebentar]","[disitunya, sebentar, kenangannya, sebentar]","[situ, sebentar, kenang, sebentar]",situ sebentar kenang sebentar,"{'situ': 0.0, 'sebentar': 0.0, 'kenang': 0.357..."


In [ ]:
sentence = 'tempatnya jauh'
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: neutral


In [ ]:
X = dflabel['text_final']
y = dflabel['sentiment']

tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.1, random_state=1, shuffle=True)

smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print(y_train_resampled.value_counts())
print('\n')
nb_classifier_smote = MultinomialNB()
nb_classifier_smote.fit(X_train_resampled, y_train_resampled)

# Predict labels for the test data
y_pred_resampled = nb_classifier_smote.predict(X_train_resampled)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_train_resampled, y_pred_resampled)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_train_resampled, y_pred_resampled))


sentiment
neutral     1201
negative    1201
positive    1201
Name: count, dtype: int64


Accuracy: 0.890646683319456
Classification Report:
              precision    recall  f1-score   support

    negative       0.85      0.96      0.91      1201
     neutral       0.93      0.82      0.87      1201
    positive       0.90      0.89      0.89      1201

    accuracy                           0.89      3603
   macro avg       0.89      0.89      0.89      3603
weighted avg       0.89      0.89      0.89      3603



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1, shuffle=True)

smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print(y_train_resampled.value_counts())
print('\n')
nb_classifier_smote = MultinomialNB()
nb_classifier_smote.fit(X_train_resampled, y_train_resampled)

# Predict labels for the test data
y_pred_resampled = nb_classifier_smote.predict(X_train_resampled)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_train_resampled, y_pred_resampled)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_train_resampled, y_pred_resampled))


sentiment
positive    1067
neutral     1067
negative    1067
Name: count, dtype: int64


Accuracy: 0.8953452046235552
Classification Report:
              precision    recall  f1-score   support

    negative       0.86      0.96      0.91      1067
     neutral       0.92      0.84      0.88      1067
    positive       0.92      0.89      0.90      1067

    accuracy                           0.90      3201
   macro avg       0.90      0.90      0.89      3201
weighted avg       0.90      0.90      0.89      3201



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=1, shuffle=True)

smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print(y_train_resampled.value_counts())
print('\n')
nb_classifier_smote = MultinomialNB()
nb_classifier_smote.fit(X_train_resampled, y_train_resampled)

# Predict labels for the test data
y_pred_resampled = nb_classifier_smote.predict(X_train_resampled)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_train_resampled, y_pred_resampled)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_train_resampled, y_pred_resampled))


sentiment
positive    935
neutral     935
negative    935
Name: count, dtype: int64


Accuracy: 0.8962566844919786
Classification Report:
              precision    recall  f1-score   support

    negative       0.86      0.97      0.91       935
     neutral       0.94      0.82      0.87       935
    positive       0.91      0.90      0.90       935

    accuracy                           0.90      2805
   macro avg       0.90      0.90      0.90      2805
weighted avg       0.90      0.90      0.90      2805



In [ ]:
sentence = "keren banget"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: positive


In [ ]:
sentence = "kemarin di sini beli ayam goreng"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: neutral


In [ ]:
sentence = "Panas banget"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: negative


In [ ]:
sentence = "aku rindu jogja"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: positive


In [ ]:
sentence = "tempatnya jauh"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: negative


In [ ]:
sentence = "Lestarikan budaya indonesia"
sentence_vectorized = tfidf_vectorizer.transform([sentence])
prediction = nb_classifier_smote.predict(sentence_vectorized)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: positive


In [ ]:
import pickle
with open('./drive/MyDrive/TA Tugas Akhir Workspace/naive_bayes_model.pkl', 'wb') as file:
    pickle.dump(nb_classifier_smote, file)
with open('./drive/MyDrive/TA Tugas Akhir Workspace/tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

In [ ]:
import joblib
joblib.dump(nb_classifier_smote, './drive/MyDrive/TA Tugas Akhir Workspace/naive_bayes_model.joblib')
joblib.dump(tfidf_vectorizer, './drive/MyDrive/TA Tugas Akhir Workspace/tfidf_vectorizer.joblib')


['./drive/MyDrive/TA Tugas Akhir Workspace/tfidf_vectorizer.joblib']

In [ ]:
directory = './drive/MyDrive/TA Tugas Akhir Workspace/Dataset/Test'
csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
csv_files

['./drive/MyDrive/TA Tugas Akhir Workspace/Dataset/Test/ytprambanan.csv',
 './drive/MyDrive/TA Tugas Akhir Workspace/Dataset/Test/Tiktok_Malioboro1.csv']

In [ ]:
import pickle
with open('./drive/MyDrive/TA Tugas Akhir Workspace/naive_bayes_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
def sentiment_predict(text, model, vectorizer):
    # text_vectorized = tfidf_vectorizer.transform([text])
    # sentiment_result = model.predict(text_vectorized)
    # return sentiment_result[0]
    text_vectorized = vectorizer.transform([text.lower().strip()])
    sentiment = model.predict(text_vectorized)[0]
    return sentiment




In [ ]:
def apply_sentiment(df, model, vectorizer):
    sentiment_column = []
    for _, row in df.iterrows():
      if 'Text' in row:
            text = row['Text']
      elif 'textOriginal' in row:
          text = row['textOriginal']
      sentiment = sentiment_predict(text, model, vectorizer)
      sentiment_column.append(sentiment)
    df['sentiment'] = sentiment_column
    return df

In [ ]:
for file in csv_files:
    # Read the current CSV file into a DataFrame
    df = pd.read_csv(file)

    # Apply sentiment prediction using the defined function
    df = apply_sentiment(df, model, tfidf_vectorizer)

    # Save the updated DataFrame to a new CSV file
    output_file = os.path.join(directory, 'updated_' + os.path.basename(file))
    df.to_csv(output_file, index=False)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1, shuffle=True)

# smote = SMOTE(random_state=42)

# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
# print(y_train_resampled.value_counts())
# nb_classifier_smote = MultinomialNB()
# nb_classifier_smote.fit(X_train_resampled, y_train_resampled)

# # Predict labels for the test data
# y_pred_train = nb_classifier_smote.predict(X_train_resampled)

# # Predict labels for the test data
# y_pred_test = nb_classifier_smote.predict(X_test)

# # Evaluate the performance of the classifier
# train_accuracy = accuracy_score(y_train_resampled, y_pred_train)
# print("Training Accuracy:", train_accuracy)

# # Evaluate the performance of the classifier on the test data
# test_accuracy = accuracy_score(y_test, y_pred_test)
# print("Test Accuracy:", test_accuracy)

# # Print classification report for the resampled training data
# print("Classification Report (Training Data):")
# print(classification_report(y_train_resampled, y_pred_train))

# # Print classification report for the test data
# print("Classification Report (Test Data):")
# print(classification_report(y_test, y_pred_test))
